# N puzzle

Specifications of the problem : [https://en.wikipedia.org/wiki/15_puzzle](https://en.wikipedia.org/wiki/15_puzzle)

## Librairies

Here's all the librairies needed

In [1]:
from random import choice
from dataclasses import dataclass, field
import numpy as np

## Initialization

We first define the puzzle dimensions, and create a structure to store the possibles actions.

In [2]:
PUZZLE_DIM = 2

@dataclass
class Action:
    src: int
    dst: int

We then have to define two important functions:
- One that return all the possible actions for a given state.
- One that perform an action

Since those functions will be called many times, we have to optimize them.

In [3]:
def available_actions(state: np.ndarray) -> list[Action]:
    """Returns all the possible actions for a given state"""
    x, y = np.argwhere(state == 0)[0] # Get the indexes of the 0 in the puzzle

    # Add all possible actions by checking boundaries
    actions = list()
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    """Returns a new state that perform the given action on a given state"""
    new_state = state.copy()
    new_state[action.src], new_state[action.dst] = new_state[action.dst], new_state[action.src]
    return new_state

Finally, we create the problem, starting from the solution we randomize it many times.

In [4]:
RANDOMIZE_STEPS = 100_000
SOLUTION = np.ndarray([])

def init_problem() -> np.ndarray[np.ndarray[int]]:
    """Return a N-puzzle problem and its solution where N is given by PUZZLE_DIM"""
    solution = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)) # Create matrix

    # Randomize it
    state = solution.copy()
    for r in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))

    return state, solution

state, SOLUTION = init_problem()

## First naive approach

For the first approach, I just wanted something that works, no matter the efficiency of the algorithm.

I implemented a simple naive Depth First Search.

In [5]:
def n_puzzle_naive(state: np.ndarray[np.ndarray[int]]) -> np.ndarray[np.ndarray[int]] :
    """Return the solution of an N puzzle for a given initial state"""
    frontier = list()
    while state is not None and not np.array_equal(SOLUTION, state):
        # Append nodes made by possibles actions in the frontier for the current state
        for action in available_actions(state):
            new_state = do_action(state, action)
            frontier.append(new_state)
        
        # Get the last state from the frontier (DFS)
        state = frontier.pop()
    return state

We won't run it, since it can be very very long. But we have a starting point.

In [6]:
# final_state = n_puzzle_naive(state.copy())

The problem is apparent: adding the same state to the frontier repeatedly makes the algorithm perform the same searches.

## Depth First Search with explored

The simplest way to solve it is to add a set that keeps track of the explored state, avoiding re-searching from them.

Before, let's implement a structure that simply stores the current state and steps in order to compute the efficiency. In the algorithm, we will also have a counter of all the visited nodes.

In [7]:
@dataclass
class Node:
    state: np.ndarray[np.ndarray[int]]
    step: int = 0

To handle the explored states, we will use a set and map the NumPy matrix into a hashable tuple in order to see if it is already in the set.

In [8]:
def n_puzzle_dfs(state: np.ndarray[np.ndarray[int]]) -> tuple[np.ndarray[np.ndarray[int]], int, int] :
    """Return the solution of an N puzzle for a given initial state"""
    frontier: list[Node] = [Node(state=state)] # heap of nodes to explore
    explored = set()    # Set of explored states
    cost = 0

    while frontier:
        cost += 1

        # Get the last state from the frontier (DFS)
        current_node = frontier.pop()

        # Goal function
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        # Explore all possible actions
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)

            # Check for state already explored
            new_tuple = tuple(map(tuple, new_state))
            if new_tuple in explored:
                continue
            explored.add(new_tuple)

            frontier.append(Node(new_state, current_node.step + 1))

    return None, None, None

Let's run the algorithm and see the results.

In [9]:
# Perform algorithm
final_state, quality, cost = n_puzzle_dfs(state.copy())

# Show results
if final_state is None:
    print("No solutions founded")
else:
    print("Quality of the solution :", quality, ", cost of the solution : ", cost)

Quality of the solution : 10 , cost of the solution :  12


As we can see this is way better.

We can try to increase the problem size.

In [10]:
PUZZLE_DIM = 3
state, SOLUTION = init_problem()

# Perform algorithm
final_state, quality, cost = n_puzzle_dfs(state.copy())

# Show results
if final_state is None:
    print("No solutions founded")
else:
    print("Quality of the solution :", quality, ", cost of the solution : ", cost)

Quality of the solution : 12430 , cost of the solution :  12776


Still very efficient for 3-puzzle, but not for 4-puzzle.

## Breadth First Search

We can simply tweak a bit the algorithm to implement Bread First Search (BFS). We should increase the quality of the solution (decrease the value), but also the cost. Indeed, this is a tradeoff between quality and cost.

In [11]:
def n_puzzle_bfs(state: np.ndarray[np.ndarray[int]]) -> tuple[np.ndarray[np.ndarray[int]], int, int] :
    """Return the solution of an N puzzle for a given initial state"""
    frontier: list[Node] = [Node(state=state)] # heap of nodes to explore
    explored = set()    # Set of explored states
    cost = 0

    while frontier:
        cost += 1

        # Get the first state from the frontier (BFS)
        current_node = frontier.pop(0)

        # Goal function
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        # Explore all possible actions
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)

            # Check for state already explored
            new_tuple = tuple(map(tuple, new_state))
            if new_tuple in explored:
                continue
            explored.add(new_tuple)

            frontier.append(Node(new_state, current_node.step + 1))

    return None, None, None

Let's run the algorithm and see the results.

In [12]:
# Perform algorithm
final_state, quality, cost = n_puzzle_bfs(state.copy())

# Show results
if final_state is None:
    print("No solutions founded")
else:
    print("Quality of the solution :", quality, ", cost of the solution : ", cost)

Quality of the solution : 24 , cost of the solution :  139344


As expected the value of the quality is minimized while the cost goes up.

The problem of the solutions is that they are informed strategies, so every state has the same weight.

In reality, we know that in the case of the n-puzzle, we can say that one state is closer than another one of the solution.

## Greedy Best First

A solution can be to have a sort of ordering inside our frontier.

So first, we need a function that estimate the quality of a state of the solution. Let's use manhattan distance.

Let's precompute the target positions of the solutions in order to optimize a bit the computation of manhattan distance.

In [13]:
def precompute_target_positions() -> dict[int, tuple[int, int]]:
    """Return a dictionary that contains the target indexes of one entry of the n-puzzle"""
    target_positions = {}
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            target_positions[SOLUTION[x, y]] = (x, y)
    return target_positions

TARGET_POSITIONS = precompute_target_positions()

In [14]:
def manhattan_distance(state: np.ndarray) -> int:
    """Compute the manhattan distance bewteen a state and the solution"""
    total_distance = 0

    # Sum each difference of distances
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue  # Skip tile 0
            target_x, target_y = TARGET_POSITIONS[value]
            total_distance += abs(target_x - x) + abs(target_y - y)

    return total_distance

We also need to update our Node structure in order to include a priority field that will contain the manhattan distance.

In [15]:
@dataclass(order=True)
class Node:
    priority: int
    state: list[list[int]] = field(compare=False)
    step: int = field(compare=False)

Finally let's implement our algorithm, it will be pretty similar to the previous one but the frontier will be a priority queue implemented with the heapq module.

In [16]:
from heapq import heappop, heappush

def n_puzzle_gbf(state):
    """Return the solution of an N puzzle for a given initial state"""
    frontier: list[Node] = [] # Use a heap for storing the states
    heappush(frontier, Node(manhattan_distance(state), state, 0))
    explored = set()
    cost = 0

    while frontier:
        cost += 1
        current_node = heappop(frontier) # Get the node with the maximum expected value

        # Goal test
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        # Explore all possible actions
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)

            # Check for state already explored
            new_tuple = tuple(map(tuple, new_state))
            if new_tuple in explored:
                continue
            explored.add(new_tuple)

            heappush(frontier, Node(manhattan_distance(new_state), new_state, current_node.step + 1))

    return None, None, cost


Let's run it and see the results.

In [17]:
# Perform algorithm
final_state, quality, cost = n_puzzle_gbf(state.copy())

# Show results
if final_state is None:
    print("No solutions founded")
else:
    print("Quality of the solution :", quality, ", cost of the solution : ", cost)

Quality of the solution : 74 , cost of the solution :  195


The quality is worse than the BFS one, but the cost is way lower than the BFS and DFS algorithms.

We can try to increase the problem size and see what's happening. At the size of seven, the algorithm can be very long, it depends on the initial shuffling. From eight, the algorithm is too long to be run.

In [18]:
for i in range(4, 7):
    PUZZLE_DIM = i
    state, SOLUTION = init_problem()
    TARGET_POSITIONS = precompute_target_positions()

    # Perform algorithm
    final_state, quality, cost = n_puzzle_gbf(state.copy())

    # Show results
    if final_state is None:
        print("No solutions founded")
    else:
        print("Puzzle dimensions :", i, "\n\tquality of the solution :", quality, "\n\tcost of the solution :", cost)

Puzzle dimensions : 4 
	quality of the solution : 288 
	cost of the solution : 3263
Puzzle dimensions : 5 
	quality of the solution : 440 
	cost of the solution : 2074
Puzzle dimensions : 6 
	quality of the solution : 2728 
	cost of the solution : 126306


## A*

As we did for DFS and BFS. We can implement A* in order to increase the quality of the solution, but the cost will also increase.

The implementation will be straightforward since A* only add the actual cost to compute the priority of a node.

In [19]:
from heapq import heappop, heappush

def n_puzzle_a_star(state):
    """Return the solution of an N puzzle for a given initial state"""
    frontier: list[Node] = [] # Use a heap for storing the states
    heappush(frontier, Node(manhattan_distance(state), state, 0))
    explored = set()
    cost = 0

    while frontier:
        cost += 1
        current_node = heappop(frontier) # Get the node with the maximum expected value

        # Goal test
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        # Explore all possible actions
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)

            # Check for state already explored
            new_tuple = tuple(map(tuple, new_state))
            if new_tuple in explored:
                continue
            explored.add(new_tuple)

            # Add the new state to the frontier
            new_step =  current_node.step + 1
            heappush(frontier, Node(manhattan_distance(new_state) + new_step, new_state, new_step)) # Add actual cost to the priority of the node

    return None, None, cost


Let's run it and see what's happening.

At the size of four, the algorithm can be very long, it depends on the initial shuffling. From five, the algorithm is too long to be run.

In [20]:
for i in range(3, 5):
    PUZZLE_DIM = i
    state, SOLUTION = init_problem()
    TARGET_POSITIONS = precompute_target_positions()

    # Perform algorithm
    final_state, quality, cost = n_puzzle_a_star(state.copy())

    # Show results
    if final_state is None:
        print("No solutions founded")
    else:
        print("Puzzle dimensions :", i, "\n\tquality of the solution :", quality, "\n\tcost of the solution :", cost)

Puzzle dimensions : 3 
	quality of the solution : 20 
	cost of the solution : 415
Puzzle dimensions : 4 
	quality of the solution : 46 
	cost of the solution : 198681


As expected, the quality of the solution increases a lot (the value goes down), but the cost also goes up.

To conclude I would choose either the A* algorithm or GBF to solve this problem. Both are good, it depends on what we want as a result.